In [1]:
import pandas as pd
import os 
#df= pd.read_csv('./sales_data/Sales_April_2019.csv')

#### Merge 12 month sales data into a single dataframe


In [2]:

files = [file for file in os.listdir('./sales_data')]

all_months_data= pd.DataFrame()  

for file in files:
  #  print(file)
    if file.endswith(".csv"):
        file_name ="./sales_data/"+file
        #print(file_name)
        df= pd.read_csv("./sales_data/"+file)
        all_months_data=pd.concat([all_months_data,df])

In [3]:
all_months_data.describe()



#### Convert the data into csv

In [4]:
all_months_data.to_csv("all_data.csv",index=False)

In [5]:
all_data=pd.read_csv("all_data.csv")

In [6]:
all_data.head()

## 10 Whats was the best months for sales? how much earned that month?

#### Augument the with addidtional columns

#### 2: Add Month column

In [7]:
all_data['Month']= all_data['Order Date'].str[0:2]

In [8]:
all_data['Month']= all_data['Month'].astype('int32')
all_data.head()

#### To resolve the above Error we need to clean the data
#### Drop the NANs

In [9]:
all_data.isna()

In [10]:
nan_all_data= all_data[all_data.isna().any(axis=1)]

In [11]:
nan_all_data.head()

In [14]:
nan_all_data.count()

In [12]:
all_data['Order ID'].index

In [13]:
all_data.iloc[199]

In [104]:
all_data.iloc[199]['Order ID']

In [105]:
all_data['Order ID'].values[199]

In [106]:
all_data.iat[199,5]

In [14]:
all_data=all_data.dropna(how="all")

In [15]:
all_data.isna()

In [16]:
temp_df= all_data[all_data['Order Date'].str[0:2]=='Or']


In [17]:
temp_df.head()

#### Some reason these records are there , we need to remove those records

In [18]:
all_data= all_data[all_data['Order Date'].str[0:2]!='Or']

In [19]:
all_data['Month']= all_data['Month'].astype('int32')
all_data.head()

#### Add a sales columns

In [20]:
all_data['Sales']= all_data['Quantity Ordered']*all_data['Price Each']

In [21]:
all_data['Quantity Ordered']= pd.to_numeric(all_data['Quantity Ordered'])

In [22]:
all_data['Price Each']= pd.to_numeric(all_data['Price Each'])

In [23]:
all_data['Sales']= all_data['Quantity Ordered']*all_data['Price Each']

In [24]:
all_data.head()

In [25]:
all_data.groupby('Month').sum()

In [26]:
all_data.head()

In [27]:
all_data.groupby('Month').agg({'Sales': {lambda x: x.nlargest(2).sum() }})

In [28]:
all_data.groupby('Month').sum().max()

In [25]:
all_data[:].groupby('Month')['Sales'].sum().max()

In [29]:
all_data.groupby('Month').sum()

In [30]:
temp_data=all_data.groupby('Month')['Sales'].sum()
temp_data.head()
temp_data.nlargest(1)

In [31]:
 all_data.loc[all_data.groupby('Month')['Sales'].sum().idxmax()]

In [29]:
 all_data.loc[all_data.groupby('Month')['Sales'].sum().idxmax()]

In [33]:
 all_data.loc[all_data.groupby('Month')['Sales'].sum().idxmax()]
results =  all_data.groupby('Month').sum() 

In [34]:
all_data.groupby('Month').sum() 


In [35]:

import matplotlib.pyplot as plt
months= range(1,13)
plt.bar(months,results['Sales'])
plt.xticks(months)
plt.ylabel('sales in usd ($)')
plt.xlabel('Month Number')
plt.show()

In [39]:
#### What city had the hightest number of sales

In [36]:
all_data['City']=all_data['Purchase Address'].apply(lambda x:x.split(',')[1])

In [37]:
all_data.head()

In [38]:
####Another way to write lambda functions
def get_city(address):
    return address.split(',')[1]

In [39]:
all_data['City']=all_data['Purchase Address'].apply(lambda x:get_city(x))

In [74]:
all_data.head()

In [ ]:
#### as  we may duplicates cities to aggreate on we need to get the state fom address


In [40]:
def get_state(address):
    return address.split(',')[2].split(' ')[1]

In [42]:
all_data['City']=all_data['Purchase Address'].apply(lambda x:get_city(x) + ' ' +get_state(x))

In [87]:
all_data.head()

In [43]:
results =  all_data.groupby('City').sum() 

In [89]:
results

In [44]:
import matplotlib.pyplot as plt
cities= all_data['City'].unique()
plt.bar(cities,results['Sales'])
plt.xticks(cities,rotation='vertical',size =8)
plt.ylabel('sales in usd ($)')
plt.xlabel('Cities')
plt.show()

### when you plot the data seems like data is wrong as Sf has more sales than Austin. this is because when we
### Unique the city order didn't change relative to sales , so we need to retain city order in lieu with sales

In [45]:
import matplotlib.pyplot as plt
cities= [city for city,df in all_data.groupby('City')]
plt.bar(cities,results['Sales'])
plt.xticks(cities,rotation='vertical',size =8)
plt.ylabel('sales in usd ($)')
plt.xlabel('Cities')
plt.show()

## What time should we display advertisements to maximize likelyhood of customer buying product

In [46]:
all_data.head()

In [47]:
all_data['Order Date']=pd.to_datetime(all_data['Order Date'])

In [48]:
all_data.head()

In [52]:
all_data['Hour']=all_data['Order Date'].dt.hour
all_data['Minute']=all_data['Order Date'].dt.minute

In [53]:
all_data.head()

In [51]:
all_data.groupby('Hour').sum()

In [58]:
hours= [hours for hours,df in all_data.groupby('Hour')]
hour_count = all_data.groupby('Hour').count()
plt.plot(hours,hour_count)
plt.xlabel('Hours')
plt.ylabel('Hourly sale count')
plt.xticks(hours)
plt.grid()
plt.show()

In [59]:
#So from the graph Recommendation is 11 am or 7 pm
#you can Break it down furthur to cities ans play to together

#### What Products most often sold together?

In [66]:
# To about this if you check the data if the order id same for multiple products that means they sold together
df= all_data[all_data['Order ID'].duplicated(keep=False)]

In [72]:
df.count()

In [75]:
df['Grouped']= df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

In [76]:
df.head()

In [78]:
## Drop duplicate
df= df[['Order ID','Grouped']].drop_duplicates()

In [79]:
df.head()

In [81]:
#Now we need iterate through the pairs and get counts
from itertools import combinations
from collections import Counter

In [89]:
count = Counter()
# Most common 2
for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))
count.most_common(10)
for key, value in count.most_common(10):
    print(key,value)
#print(count)       

# Most common 3

In [88]:
# Most common 3
count = Counter()
for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,3)))
count.most_common(10)
for key, value in count.most_common(10):
    print(key,value)

### What Product sold the most and why it sold the most?

In [90]:
# group by prodcuts and sum the quantity

In [91]:
product_sold =all_data.groupby('Product')

In [92]:
product_sold.sum()

In [94]:
quantity_ordered = product_sold.sum()['Quantity Ordered']

In [101]:
product_sold =all_data.groupby('Product')
quantity_ordered = product_sold.sum()['Quantity Ordered']
products= [products for products,df in product_sold]
plt.bar(products,quantity_ordered)
plt.xticks(products,rotation='vertical',size =10)

plt.show()

In [103]:
prices =all_data.groupby('Product').mean()['Price Each']
prices

In [108]:
#now let's combine this data to same plot
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

ax1.bar(products,quantity_ordered,color='g')
ax2.plot(products,prices,'b-')

ax1.set_xlabel('prodcuts')
ax1.set_ylabel('Quantity ordered', color='g')
ax2.set_ylabel('Price Each', color='b')
ax1.set_xticklabels(products,rotation='vertical',size=8)
plt.show()

In [109]:
#Based on the above graph , we can correlate price and quantity ordered , in above lower price and more orders.